<a href="https://colab.research.google.com/github/hfarley/bigdata-finalproject/blob/main/BigDataFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [7]:
#install spark. we are using the one that uses hadoop as the underlying scheduler.
# !rm spark-3.2.4-bin-hadoop3.2.tgz 
!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf  spark-3.2.4-bin-hadoop3.2.tgz
!ls -l

#Provides findspark.init() to make pyspark importable as a regular library.
os.environ["SPARK_HOME"] = "spark-3.2.4-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 pyspark-shell'

total 294136
drwxr-xr-x  1 root root      4096 Apr 21 13:38 sample_data
drwxr-xr-x 13 1000 1000      4096 Apr  9 21:17 spark-3.2.4-bin-hadoop3.2
-rw-r--r--  1 root root 301183180 Apr  9 21:35 spark-3.2.4-bin-hadoop3.2.tgz


In [8]:
!pip install -q findspark pyspark
import findspark
findspark.init()

In [ ]:
# 1. query any month of any year within the database and it will show a map of the incidents during that month. 
# 2. input a road segment, and it will show a graph of the density of the times 
# that incidents occur in that road segment. 
# 3. query the average response time for each road segment, and sort it to find
#  which road segments have the highest and lowest response times. 



In [12]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName("read_parquet").getOrCreate()

df = spark.read.parquet("nfd_incidents_xd_seg.parquet")

# create a smaller DataFrame with the first 1000 entries
df_small = df.limit(1000)

# write the output to use for testing/debugging
df_small.write.parquet("incidents_1000.parquet")

Notes: 


*   .show() will give the first 20 entries
*   in the spark assignment we only used SparkContext, but here we also need Spark Session

For the first task we need ID, latitude & longitude (or, alternatively geometry), and one of the time variables - we need to decide which we will use to seperate by month

Need to decide how we want to visualize this and get the coordinates to draw a map of Davidson county which then shows the incidents



In [15]:
%%file month_incidents.py



from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

if __name__ == '__main__':
  # replace this line with the s3 pass when testing over EMR
  conf = SparkConf().setAppName('month_incidents').set('spark.hadoop.validateOutputSpecs', False)
  sc = SparkContext(conf=conf).getOrCreate()
  spark = SparkSession(sc)


  try:
    incidents = spark.read.parquet("incidents_1000.parquet")
    
    incidents.show()

    
    # counts.repartition(1).saveAsTextFile("")

  finally:
    # very important: stop the context. Otherwise you may get an error that context is still alive. if you are on colab just restart the runtime if you face problem
    #finally is used to make sure the context is stopped even with errors
    sc.stop()

  pass

Overwriting month_incidents.py


In [ ]:
#to test locally
!spark-submit --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 month_incidents.py

In [ ]:
%%file road_density_incidents.py

from pyspark import SparkContext, SparkConf
import json
from operator import add

In [ ]:
!spark-submit --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 road_density_incidents.py

In [ ]:
%%file road_avg_response.py

from pyspark import SparkContext, SparkConf
import json
from operator import add

In [ ]:
!spark-submit --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.7 road_avg_response.py